In [1]:
import pandas as pd
import numpy as np
import re
import warnings
import matplotlib
import matplotlib.pyplot as plt
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import os

In [2]:
pd.set_option('display.max_colwidth', 255)
df_train =pd.read_csv('Urdu-threat-detection-FIRE2022/Train_Clean.csv')
df_test=pd.read_csv('Urdu-threat-detection-FIRE2022/Test_Clean.csv')
df_train

,Tweets,label,S/G,word_count,char_count,word_density
0,بکواس مت,1,1,2,7,0.250000
1,تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو,1,0,9,37,0.236842
2,آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے,1,1,11,52,0.207547
3,غدار منافق میر اللہ تمہیں زلیل,1,1,6,25,0.230769
4,اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے,1,0,23,83,0.273810
...,...,...,...,...,...,...
3559,ارے شعر حقیقت میں پیچھے نا پڑنا,0,2,7,25,0.269231
3560,ہندوستانی باوردی غنڈا ایل او سی پتھر مار چیک کر رہا تھا پاکستانی فوجی جاگ یا سو,0,2,17,63,0.265625
3561,خبر افسوس ناک ھے,0,2,4,13,0.285714
3562,جناب محترم سلمان وفادارصاھب کیا پارلیمنٹ آئینی ادارہ نہیں جس اینٹ سے اینٹ بجانے والا شخص,0,2,16,73,0.216216


In [3]:
df_test

,Tweet,Threat,G/S
0,وزیراعل پنجاب کا ضلع راولپنڈی کا دورہ لینڈسلائیڈنگ متاثرین سے سمری طلب ہوٹل انڈسٹری پیکج کا اعلان,0,2
1,الحمدللہ پورے پاکستان وکلاء برادری کا کہنا ہم ضامن مراثی خلاف فری میں کیس لڑیں میں ان تمام وکلاء کا بہت شکر گزار اللہ تعالی آپ کا حامی وناصر آپ اس جر کو سلام دشمن اصحاب رسول تیری اب خیر نہیں معاویہ کوگرفتارکرو پرلعن,1,1
2,جنوری مہلت ختم الٹی گنتی شروع ن لیگ خلاف بڑی کارروائی کا فیصلہ,1,1
3,کروناکی آڑ میں حکومت اورفوج جوکھیل قادیانیوں کیلیےکھیل وہ ہرگز اس بھول میں مت رہیں وہ اس مقصد میں کامیاب ہم نے میں بھی شہادتیں بھی گردنیں حضور لیے ہوٸیں شہید یاان زندیقوں کو کاٹ دیں ان شاء اللہ,1,1
4,حکومت ایکشن میں ورنہ اگر ہم ایکشن میں آگئے ہم کسی کوبھی نہیں چھوڑیں,1,1
...,...,...,...
930,کوئ پیڈ جیالوں کو بتا عمران احمد خان نیازی نام سے گالی کا ٹرینڈ کیا جواب میں ان کتوں شلواریں پینل ٹانگ دیں اس بات میں کسی کو کوئ شک نہیں ہونا چاہیے,1,1
931,مجبور نہ قانون ہاتھ میں دین اسلام خلاف بھی قانون گا اس کو میں جوتے نوک رکھتا زندگی لاکھ بھی دین قربان کر بھی دل نا بھرتا ہماری پاک آرمی ایسے کو کنٹرول جو دین اسلام خلاف کام کر,1,1
932,ڈرامہ سیاست زلیل گہٹیا غلطی نہیں مانیں,0,2
933,وزیراعظم صاحب آپ کا اتنی جلد بازی کا فیصلہ آپ جہالت کا م نہ بولتا ثبوت,0,2


In [4]:
#dropping the column word_count ,char_count and word_density
df_train_copy=df_train.copy()
df_train_copy.drop(["word_count","char_count","word_density"],axis=1,inplace=True)
df_train_copy

,Tweets,label,S/G
0,بکواس مت,1,1
1,تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو,1,0
2,آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے,1,1
3,غدار منافق میر اللہ تمہیں زلیل,1,1
4,اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے,1,0
...,...,...,...
3559,ارے شعر حقیقت میں پیچھے نا پڑنا,0,2
3560,ہندوستانی باوردی غنڈا ایل او سی پتھر مار چیک کر رہا تھا پاکستانی فوجی جاگ یا سو,0,2
3561,خبر افسوس ناک ھے,0,2
3562,جناب محترم سلمان وفادارصاھب کیا پارلیمنٹ آئینی ادارہ نہیں جس اینٹ سے اینٹ بجانے والا شخص,0,2


In [5]:
df_train_copy.rename(columns={"Tweets":"Tweet","label":"Threat","S/G":"G/S"},inplace=True)
df_train_copy

,Tweet,Threat,G/S
0,بکواس مت,1,1
1,تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو,1,0
2,آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے,1,1
3,غدار منافق میر اللہ تمہیں زلیل,1,1
4,اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے,1,0
...,...,...,...
3559,ارے شعر حقیقت میں پیچھے نا پڑنا,0,2
3560,ہندوستانی باوردی غنڈا ایل او سی پتھر مار چیک کر رہا تھا پاکستانی فوجی جاگ یا سو,0,2
3561,خبر افسوس ناک ھے,0,2
3562,جناب محترم سلمان وفادارصاھب کیا پارلیمنٹ آئینی ادارہ نہیں جس اینٹ سے اینٹ بجانے والا شخص,0,2


In [6]:
# to remove transcription rows that is empty
df_train_copy = df_train_copy[df_train_copy['Tweet'].notna()]
df_test=df_test[df_test["Tweet"].notna()]
df_train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3564 entries, 0 to 3563
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   3564 non-null   object
 1   Threat  3564 non-null   int64 
 2   G/S     3564 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 111.4+ KB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 935 entries, 0 to 934
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   935 non-null    object
 1   Threat  935 non-null    int64 
 2   G/S     935 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 29.2+ KB


After dropping the null values, there are no null values for the tweets attribute. 

Data normalisation will be conducted for the trascription. One of the reasons is to convert the transcript into standard format, which important for data extraction later. In this data normalisation task, following task will be executed, which are:
1. Lowe Case
2. Removing punctuation and numbers
3. Tokenisation of the transcription
4. Lemmatisation
5. Remove Stop Words

In [8]:
df_train_copy.to_csv("Modified_Clean_train.csv")
df_train=pd.read_csv("Modified_Clean_train.csv",index_col=0)
df_train

,Tweet,Threat,G/S
0,بکواس مت,1,1
1,تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو,1,0
2,آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے,1,1
3,غدار منافق میر اللہ تمہیں زلیل,1,1
4,اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے,1,0
...,...,...,...
3559,ارے شعر حقیقت میں پیچھے نا پڑنا,0,2
3560,ہندوستانی باوردی غنڈا ایل او سی پتھر مار چیک کر رہا تھا پاکستانی فوجی جاگ یا سو,0,2
3561,خبر افسوس ناک ھے,0,2
3562,جناب محترم سلمان وفادارصاھب کیا پارلیمنٹ آئینی ادارہ نہیں جس اینٹ سے اینٹ بجانے والا شخص,0,2


In [9]:
#concatinating the dataframes to and then we will slice them later for train and test data 
df=pd.concat([df_train,df_test],ignore_index=True)
df

,Tweet,Threat,G/S
0,بکواس مت,1,1
1,تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو,1,0
2,آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے,1,1
3,غدار منافق میر اللہ تمہیں زلیل,1,1
4,اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے,1,0
...,...,...,...
4494,کوئ پیڈ جیالوں کو بتا عمران احمد خان نیازی نام سے گالی کا ٹرینڈ کیا جواب میں ان کتوں شلواریں پینل ٹانگ دیں اس بات میں کسی کو کوئ شک نہیں ہونا چاہیے,1,1
4495,مجبور نہ قانون ہاتھ میں دین اسلام خلاف بھی قانون گا اس کو میں جوتے نوک رکھتا زندگی لاکھ بھی دین قربان کر بھی دل نا بھرتا ہماری پاک آرمی ایسے کو کنٹرول جو دین اسلام خلاف کام کر,1,1
4496,ڈرامہ سیاست زلیل گہٹیا غلطی نہیں مانیں,0,2
4497,وزیراعظم صاحب آپ کا اتنی جلد بازی کا فیصلہ آپ جہالت کا م نہ بولتا ثبوت,0,2


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4499 entries, 0 to 4498
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   4499 non-null   object
 1   Threat  4499 non-null   int64 
 2   G/S     4499 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 105.6+ KB


# <b>2.3 <span style='color:red'>|</span> Tokenisation </b> <a class="anchor" id="2.3"></a>


In [11]:
# to tokenise transcription

# import nltk
tk =WhitespaceTokenizer()
def tokenise(df, attribute):
    df['tokenised'] = df.apply(lambda row: tk.tokenize(str(row[attribute])), axis=1)
    return df
df=tokenise(df, 'Tweet')
df_experiment =df_train.copy()


In [12]:
display(df)

,Tweet,Threat,G/S,tokenised
0,بکواس مت,1,1,"[بکواس, مت]"
1,تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو,1,0,"[تمہاری, ہیجڑا, فورس, نہتے, سامنے, بکری, بنی, ڈوب, مرو]"
2,آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے,1,1,"[آفیسر, سمیتبھارتی, فوجی, جہنم, واصلبنکرز, تباہ, بھارت, کو, پتہ, لگ, جائے]"
3,غدار منافق میر اللہ تمہیں زلیل,1,1,"[غدار, منافق, میر, اللہ, تمہیں, زلیل]"
4,اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے,1,0,"[اگست, آپ, بھارت, کو, کیا, پیغام, دینا, چاہیں, میرا, پیغام, ھے, کشمیر, سے, نکل, جاو, ورنہ, کتے, موت, مارے, جاو, کشمیر, ہمارا, ھے]"
...,...,...,...,...
4494,کوئ پیڈ جیالوں کو بتا عمران احمد خان نیازی نام سے گالی کا ٹرینڈ کیا جواب میں ان کتوں شلواریں پینل ٹانگ دیں اس بات میں کسی کو کوئ شک نہیں ہونا چاہیے,1,1,"[کوئ, پیڈ, جیالوں, کو, بتا, عمران, احمد, خان, نیازی, نام, سے, گالی, کا, ٹرینڈ, کیا, جواب, میں, ان, کتوں, شلواریں, پینل, ٹانگ, دیں, اس, بات, میں, کسی, کو, کوئ, شک, نہیں, ہونا, چاہیے]"
4495,مجبور نہ قانون ہاتھ میں دین اسلام خلاف بھی قانون گا اس کو میں جوتے نوک رکھتا زندگی لاکھ بھی دین قربان کر بھی دل نا بھرتا ہماری پاک آرمی ایسے کو کنٹرول جو دین اسلام خلاف کام کر,1,1,"[مجبور, نہ, قانون, ہاتھ, میں, دین, اسلام, خلاف, بھی, قانون, گا, اس, کو, میں, جوتے, نوک, رکھتا, زندگی, لاکھ, بھی, دین, قربان, کر, بھی, دل, نا, بھرتا, ہماری, پاک, آرمی, ایسے, کو, کنٹرول, جو, دین, اسلام, خلاف, کام, کر]"
4496,ڈرامہ سیاست زلیل گہٹیا غلطی نہیں مانیں,0,2,"[ڈرامہ, سیاست, زلیل, گہٹیا, غلطی, نہیں, مانیں]"
4497,وزیراعظم صاحب آپ کا اتنی جلد بازی کا فیصلہ آپ جہالت کا م نہ بولتا ثبوت,0,2,"[وزیراعظم, صاحب, آپ, کا, اتنی, جلد, بازی, کا, فیصلہ, آپ, جہالت, کا, م, نہ, بولتا, ثبوت]"


In [13]:
df["G/S"].unique()

array([1, 0, 2])

In [14]:
df['stemmed_without_stop'] = df["tokenised"].apply(lambda x: ' '.join([word for word in x ]))

In [15]:
df

,Tweet,Threat,G/S,tokenised,stemmed_without_stop
0,بکواس مت,1,1,"[بکواس, مت]",بکواس مت
1,تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو,1,0,"[تمہاری, ہیجڑا, فورس, نہتے, سامنے, بکری, بنی, ڈوب, مرو]",تمہاری ہیجڑا فورس نہتے سامنے بکری بنی ڈوب مرو
2,آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے,1,1,"[آفیسر, سمیتبھارتی, فوجی, جہنم, واصلبنکرز, تباہ, بھارت, کو, پتہ, لگ, جائے]",آفیسر سمیتبھارتی فوجی جہنم واصلبنکرز تباہ بھارت کو پتہ لگ جائے
3,غدار منافق میر اللہ تمہیں زلیل,1,1,"[غدار, منافق, میر, اللہ, تمہیں, زلیل]",غدار منافق میر اللہ تمہیں زلیل
4,اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے,1,0,"[اگست, آپ, بھارت, کو, کیا, پیغام, دینا, چاہیں, میرا, پیغام, ھے, کشمیر, سے, نکل, جاو, ورنہ, کتے, موت, مارے, جاو, کشمیر, ہمارا, ھے]",اگست آپ بھارت کو کیا پیغام دینا چاہیں میرا پیغام ھے کشمیر سے نکل جاو ورنہ کتے موت مارے جاو کشمیر ہمارا ھے
...,...,...,...,...,...
4494,کوئ پیڈ جیالوں کو بتا عمران احمد خان نیازی نام سے گالی کا ٹرینڈ کیا جواب میں ان کتوں شلواریں پینل ٹانگ دیں اس بات میں کسی کو کوئ شک نہیں ہونا چاہیے,1,1,"[کوئ, پیڈ, جیالوں, کو, بتا, عمران, احمد, خان, نیازی, نام, سے, گالی, کا, ٹرینڈ, کیا, جواب, میں, ان, کتوں, شلواریں, پینل, ٹانگ, دیں, اس, بات, میں, کسی, کو, کوئ, شک, نہیں, ہونا, چاہیے]",کوئ پیڈ جیالوں کو بتا عمران احمد خان نیازی نام سے گالی کا ٹرینڈ کیا جواب میں ان کتوں شلواریں پینل ٹانگ دیں اس بات میں کسی کو کوئ شک نہیں ہونا چاہیے
4495,مجبور نہ قانون ہاتھ میں دین اسلام خلاف بھی قانون گا اس کو میں جوتے نوک رکھتا زندگی لاکھ بھی دین قربان کر بھی دل نا بھرتا ہماری پاک آرمی ایسے کو کنٹرول جو دین اسلام خلاف کام کر,1,1,"[مجبور, نہ, قانون, ہاتھ, میں, دین, اسلام, خلاف, بھی, قانون, گا, اس, کو, میں, جوتے, نوک, رکھتا, زندگی, لاکھ, بھی, دین, قربان, کر, بھی, دل, نا, بھرتا, ہماری, پاک, آرمی, ایسے, کو, کنٹرول, جو, دین, اسلام, خلاف, کام, کر]",مجبور نہ قانون ہاتھ میں دین اسلام خلاف بھی قانون گا اس کو میں جوتے نوک رکھتا زندگی لاکھ بھی دین قربان کر بھی دل نا بھرتا ہماری پاک آرمی ایسے کو کنٹرول جو دین اسلام خلاف کام کر
4496,ڈرامہ سیاست زلیل گہٹیا غلطی نہیں مانیں,0,2,"[ڈرامہ, سیاست, زلیل, گہٹیا, غلطی, نہیں, مانیں]",ڈرامہ سیاست زلیل گہٹیا غلطی نہیں مانیں
4497,وزیراعظم صاحب آپ کا اتنی جلد بازی کا فیصلہ آپ جہالت کا م نہ بولتا ثبوت,0,2,"[وزیراعظم, صاحب, آپ, کا, اتنی, جلد, بازی, کا, فیصلہ, آپ, جہالت, کا, م, نہ, بولتا, ثبوت]",وزیراعظم صاحب آپ کا اتنی جلد بازی کا فیصلہ آپ جہالت کا م نہ بولتا ثبوت


# <b>3.0 <span style='color:red'>|</span> Text N-Gram Feature Extraction </b> <a class="anchor" id="3.0"></a>

We will use sklearn class 'CountVectoriser' to extract different n-grams features. In order to do so, the transcription should be converted into a list format, rather than a dataframe. For the purpose of converting into a flat list (i.e., there is no inner list), the function of 'flat_list' that defined above is used.

In [16]:
# function to flatten one list
def flat_list(unflat_list):
    flatted = [item for sublist in unflat_list for item in sublist]
    return flatted

def to_list(df, attribute):
    # Select the normalised transcript column 
    df_transcription = df[[attribute]]
    # To convert the attribute into list format, but it has inner list. So it cannot put into the CountVectoriser
    unflat_list_transcription = df_transcription.values.tolist()
    # Let's use back the function defined above, "flat_list", to flatten the list
    flat_list_transcription = flat_list(unflat_list_transcription)
    return flat_list_transcription
flat_list_transcription = to_list(df ,'stemmed_without_stop')

# <b>3.1 <span style='color:red'>|</span> Extract 5 Types of N-Gram </b> <a class="anchor" id="3.1"></a>


CountVectorizer is used to convert a collection of transcript documents to a matrix of n-gram features. To explain the ngram_range, all values of n such such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.

In [17]:
n_gram_features ={'unigram':(1,1),'unigram_bigram':(1,2),'bigram':(2,2),\
       'bigram_trigram':(2,3),'trigram':(3,3)}
feature_name=[]
temp=[]
for key, values in n_gram_features.items():
    temp.append(key)
    feature_name.append(key)
temp

['unigram', 'unigram_bigram', 'bigram', 'bigram_trigram', 'trigram']

In [18]:
def generate_n_gram_features(flat_list_transcription):
    temp=[]
    for key, values in n_gram_features.items(): 
        vectorizer = CountVectorizer(ngram_range=values)
        vectorizer.fit(flat_list_transcription)
        temp.append(vectorizer.transform(flat_list_transcription))
    return temp
temp = generate_n_gram_features(flat_list_transcription)

# <b>3.2 <span style='color:red'>|</span> Dimension of Each Feature Vector </b> <a class="anchor" id="3.2"></a>

In [19]:
dataframes = {'unigram':temp[0], 
              'unigram_bigram':temp[1], 
              'bigram':temp[2], 
              'bigram_trigram':temp[3], 
              'trigram':temp[4]}
feature_vector= [] ; feature_vector_shape = []
for key in dataframes:
    feature_vector.append(key)
    feature_vector_shape.append(dataframes[key].shape)

n_gram_df = pd.DataFrame({'N-Gram Feature Vector':feature_vector, 'Data Dimension':feature_vector_shape})


In [20]:
n_gram_df

,N-Gram Feature Vector,Data Dimension
0,unigram,"(4499, 9045)"
1,unigram_bigram,"(4499, 48979)"
2,bigram,"(4499, 39934)"
3,bigram_trigram,"(4499, 88705)"
4,trigram,"(4499, 48771)"


After the feature extraction process, 5 kinds of n-gram features are extracted. It is interesting to notice that when the number of 'n' getting higher (i.e, n=1:unigram, n=2:bigram, n=3:trigram), there is a higer number of columns. This is due to it is getting harder to find similar features that can be stored in similar column when it has a longer connected words as one featuer. If the feature is unique, it will automatically append additional column to store the feaure.

In [21]:
# to retrieve a unigram feature vector
dataframes['unigram']

<4499x9045 sparse matrix of type '<class 'numpy.int64'>'
	with 58117 stored elements in Compressed Sparse Row format>

# <b>4.0 <span style='color:red'>|</span> Text Classification Modelling </b> <a class="anchor" id="4.0"></a>

In [22]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import warnings
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')
random_state_number =8888

In [24]:
df_target =df[['G/S']].values.ravel()
df_target
# get evaluation result
from sklearn.metrics import balanced_accuracy_score

In [29]:
metrics = {
    'f1':[f1_score, 'f1'], 
    'precision': [precision_score, 'precision_macro'], 
    'recall': [recall_score, 'recall_macro']
#     "accuracy":[balanced_accuracy_score,"balanced_accuracy"]
#     "roc_auc":[roc_auc_score,"roc_auc_macro"]
}


def get_performance(param_grid, base_estimator, dataframes):
    df_name_list =[]; best_estimator_list=[]; best_score_list=[]; test_predict_result_list=[];
    metric_list = [];
    
    for df_name, df in dataframes.items():
        
        X_train, X_test, y_train, y_test =  df[:3564],df[3564:],df_target[:3564],df_target[3564:]#train_test_split(df, df_train["label"], test_size=0.2, random_state=random_state_number)
        for _, metric_dict in metrics.items():
            sh = HalvingGridSearchCV(base_estimator, param_grid, cv=5, scoring=metric_dict[1],random_state=random_state_number,
                                      factor=2).fit(X_train, y_train)

            best_estimator = sh.best_estimator_
            clf = best_estimator.fit(X_train, y_train)
            prediction = clf.predict(X_test)
            test_predict_result = metric_dict[0](y_test, prediction, average='macro')#for multiclass
#             test_predict_result = metric_dict[0](y_test, prediction)#for binary

            df_name_list.append(df_name) ; best_estimator_list.append(best_estimator) ; 
            best_score_list.append(sh.best_score_) ; 
            test_predict_result_list.append(test_predict_result) ;metric_list.append(metric_dict[1])
            
            
    model_result = pd.DataFrame({'Vector':df_name_list,'Metric':metric_list,
                               'Calibrated Estimator':best_estimator_list,
                               'Best CV Metric Score':best_score_list, 'Test Predict Metric Score': test_predict_result_list})
    return model_result

In [75]:
#getting random forest classifiers results 
param_grid = {'max_depth': [None,30,32,35,37,38,39,40],'min_samples_split': [2,150,170,180,190,200]}
base_estimator = RandomForestClassifier(random_state=random_state_number)
rfc_result = get_performance(param_grid, base_estimator, dataframes)
rfc_result

,Vector,Metric,Calibrated Estimator,Best CV Metric Score,Test Predict Metric Score
0,unigram,f1,"(DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1985925507), DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1...",NaN,0.292889
1,unigram,precision_macro,"(DecisionTreeClassifier(max_features='sqrt', random_state=1985925507), DecisionTreeClassifier(max_features='sqrt', random_state=1459224502), DecisionTreeClassifier(max_features='sqrt', random_state=834170083), DecisionTreeClassifier(max_features='sqrt...",0.550217,0.326195
2,unigram,recall_macro,"(DecisionTreeClassifier(max_features='sqrt', random_state=1985925507), DecisionTreeClassifier(max_features='sqrt', random_state=1459224502), DecisionTreeClassifier(max_features='sqrt', random_state=834170083), DecisionTreeClassifier(max_features='sqrt...",0.489422,0.374357
3,unigram_bigram,f1,"(DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1985925507), DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1...",NaN,0.278173
4,unigram_bigram,precision_macro,"(DecisionTreeClassifier(max_features='sqrt', min_samples_split=150,\n random_state=1985925507), DecisionTreeClassifier(max_features='sqrt', min_samples_split=150,\n random_state=1459224502), DecisionTreeClas...",0.621319,0.255559
5,unigram_bigram,recall_macro,"(DecisionTreeClassifier(max_features='sqrt', min_samples_split=170,\n random_state=1985925507), DecisionTreeClassifier(max_features='sqrt', min_samples_split=170,\n random_state=1459224502), DecisionTreeClas...",0.444454,0.378412
6,bigram,f1,"(DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1985925507), DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1...",NaN,0.279339
7,bigram,precision_macro,"(DecisionTreeClassifier(max_features='sqrt', min_samples_split=150,\n random_state=1985925507), DecisionTreeClassifier(max_features='sqrt', min_samples_split=150,\n random_state=1459224502), DecisionTreeClas...",0.576205,0.284745
8,bigram,recall_macro,"(DecisionTreeClassifier(max_features='sqrt', min_samples_split=150,\n random_state=1985925507), DecisionTreeClassifier(max_features='sqrt', min_samples_split=150,\n random_state=1459224502), DecisionTreeClas...",0.421085,0.383834
9,bigram_trigram,f1,"(DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1985925507), DecisionTreeClassifier(max_depth=40, max_features='sqrt', min_samples_split=190,\n random_state=1...",NaN,0.288805


In [30]:
#getting KNN classifier results
param_grid = {'n_neighbors': [5,7,9,11,13,15,17,19,21]}
base_estimator = KNeighborsClassifier()
knn_result = get_performance(param_grid, base_estimator, dataframes)
knn_result

,Vector,Metric,Calibrated Estimator,Best CV Metric Score,Test Predict Metric Score
0,unigram,f1,KNeighborsClassifier(n_neighbors=19),NaN,0.303794
1,unigram,precision_macro,KNeighborsClassifier(n_neighbors=15),0.454061,0.306153
2,unigram,recall_macro,KNeighborsClassifier(),0.456626,0.285908
3,unigram_bigram,f1,KNeighborsClassifier(n_neighbors=19),NaN,0.300102
4,unigram_bigram,precision_macro,KNeighborsClassifier(n_neighbors=9),0.445006,0.420602
5,unigram_bigram,recall_macro,KNeighborsClassifier(n_neighbors=9),0.378603,0.388471
6,bigram,f1,KNeighborsClassifier(n_neighbors=19),NaN,0.334947
7,bigram,precision_macro,KNeighborsClassifier(n_neighbors=15),0.512850,0.221003
8,bigram,recall_macro,KNeighborsClassifier(n_neighbors=13),0.352802,0.282048
9,bigram_trigram,f1,KNeighborsClassifier(n_neighbors=19),NaN,0.311084


In [31]:
#getting decision tree classifier results
param_grid = {'max_depth': [None,4,6,7,8,30,32,35],'min_samples_split': [2,3,4,5,35,10,16,20]}
base_estimator = DecisionTreeClassifier(random_state=random_state_number)
dtc_result = get_performance(param_grid, base_estimator, dataframes)
dtc_result

,Vector,Metric,Calibrated Estimator,Best CV Metric Score,Test Predict Metric Score
0,unigram,f1,"DecisionTreeClassifier(max_depth=35, min_samples_split=20, random_state=8888)",NaN,0.315305
1,unigram,precision_macro,"DecisionTreeClassifier(max_depth=35, min_samples_split=16, random_state=8888)",0.467091,0.383014
2,unigram,recall_macro,"DecisionTreeClassifier(min_samples_split=3, random_state=8888)",0.455793,0.346426
3,unigram_bigram,f1,"DecisionTreeClassifier(max_depth=35, min_samples_split=20, random_state=8888)",NaN,0.311865
4,unigram_bigram,precision_macro,"DecisionTreeClassifier(max_depth=35, min_samples_split=10, random_state=8888)",0.476481,0.374896
5,unigram_bigram,recall_macro,"DecisionTreeClassifier(min_samples_split=5, random_state=8888)",0.449504,0.355257
6,bigram,f1,"DecisionTreeClassifier(max_depth=35, min_samples_split=20, random_state=8888)",NaN,0.311065
7,bigram,precision_macro,"DecisionTreeClassifier(min_samples_split=35, random_state=8888)",0.453001,0.315052
8,bigram,recall_macro,"DecisionTreeClassifier(min_samples_split=35, random_state=8888)",0.431019,0.409009
9,bigram_trigram,f1,"DecisionTreeClassifier(max_depth=35, min_samples_split=20, random_state=8888)",NaN,0.310703


In [ ]:
df_result = pd.concat([knn_result, 
                      dtc_result,
                      rfc_result
                      ]
                     ).reset_index(drop=True)

df_result.groupby(['Metric']).max()

# <b>4.3 <span style='color:red'>|</span>  Obtain Best Classifier and Feature Vector </b> <a class="anchor" id="4.3"></a>

In [ ]:
def get_best_result(df_result, metric_score):
    df_result_t = df_result[df_result.Metric== 'precision_macro']
    precision_macro_df = df_result_t.loc[df_result_t[metric_score].idxmax()].to_frame().T

    df_result_t = df_result[df_result.Metric== 'recall_macro']
    recall_macro_df = df_result_t.loc[df_result_t[metric_score].idxmax()].to_frame().T
    
    df_result_t = df_result[df_result.Metric== 'f1_macro']
    f1_macro_df = df_result_t.loc[df_result_t[metric_score].idxmax()].to_frame().T

    return pd.concat([precision_macro_df,recall_macro_df,f1_macro_df])

In [ ]:
best_cv_result = get_best_result(df_result, 'Best CV Metric Score')
display(best_cv_result)
temp = best_cv_result[best_cv_result['Metric'] == 'f1_macro']
best_clf = temp['Calibrated Estimator'].values[0]
best_vector = temp['Vector'].values[0]

In [ ]:
get_best_result(df_result, 'Test Predict Metric Score')